## Imported packages

- I update `conda` on a weekly basis, so whatever version is supplied from the distribution update

In [ ]:
import pandas as pd
from pandas.io.json import json_normalize
from PIL import Image
import pytesseract
from pdf2image import convert_from_path 
from pathlib import Path
import json
import re
from pprint import pprint as pp
from ast import literal_eval

## Pandas configuration options

In [ ]:
pd.set_option('display.max_columns', 700)
pd.set_option('display.max_rows', 100)
pd.set_option('display.min_rows', 10)
pd.set_option('display.expand_frame_repr', True)

## Config files for the location of the tesseract installation

In [ ]:
pytesseract.pytesseract.tesseract_cmd = r'E:\Program Files\Tesseract-OCR\tesseract.exe'
tessdata_dir_config = '--tessdata-dir "E:\\Program Files\\Tesseract-OCR\\tessdata"'

## Testing

In [ ]:
pdf = pytesseract.image_to_pdf_or_hocr(r'E:\Users\Trenton J. McKinney\Documents\Dropbox\D & D\2nd Ed Rules\Tome of Magic (TSR2121).pdf', extension='pdf', config=tessdata_dir_config)

In [ ]:
from skimage import io
import numpy as np

In [ ]:
images = list(Path.cwd().glob('*.png'))

In [ ]:
for image in images:
    im = io.imread(image, as_gray=True)
    im = np.invert(im)
    
#     image_text = pytesseract.image_to_string(Image.open(image), config=tessdata_dir_config)
    image_text = pytesseract.image_to_string(im, config=tessdata_dir_config)
    print(image_text)

In [ ]:
image_text = pytesseract.image_to_string(Image.open('so.png'), config=tessdata_dir_config)

In [ ]:
image_text

In [ ]:
i = image_text

In [ ]:
il = i.split('\n')

In [ ]:
il

## OCR pdf file and save text to a `.txt` file

- Copyright prevents the distribution of the source material, you'll have to possess a personal copy

In [ ]:
f = list(Path(r'E:\PythonProjects\Dungeons_and_Dragons\pdf_data').glob('bigb*.pdf'))
         
filename_list = [r'E:\PythonProjects\Dungeons_and_Dragons\pdf_data\tome_of_magic_wizard_spells_2nd.pdf',
                 r'E:\PythonProjects\Dungeons_and_Dragons\pdf_data\tome_of_magic_cleric_spells_2nd.pdf',]

for filename in f:
    text = ''
    pages = convert_from_path(filename, 500)
    text += '\n'.join(pytesseract.image_to_string(page, config=tessdata_dir_config) for page in pages)
    filename.with_suffix('.txt').write_text(text)

In [ ]:
pp(text)

## Review `spells.json` in `pandas`

In [ ]:
p = Path.cwd() / 'spells.json'

In [ ]:
with p.open('r', encoding='utf-8') as f:
    data = json.loads(f.read())

In [ ]:
df = json_normalize(data)
df

In [ ]:
for x in range(len(df)):
    print(df.Description.iloc[x])
    print('*'*100)

## Clean Description:

In [ ]:
test = """When this spell 1s employed, the wizard can
command any nonliving rope-like object, including string, yarn, cord, line, rope, or even a cable.
The spell affects 50 feet of normal rope (with a 1
inch diameter), plus 5 feet per caster level. This
length is reduced by 50% for every additional
inch of thickness and increased by 50% for each
half-inch less. The possible commands are Coil
(form a neat, coiled stack), Coil & Knot, Loop,
Loop & Knot, Tie & Knot, and the reverses of all
of the above (Uncoil, etc.). One command can be
given each round.\n\n

The rope can only enwrap a creature or an
object within 1 foot of it—it does not snake outward—so it must be thrown or hurled near the
intended target. Note that the rope itself, and any
knots tied in it, are not magical. A typical rope
might be AC 6 and take 4 points of slashing damage before breaking. The rope does not inflict
damage of any type, but it can be used as a trip
line or to entangle a single opponent who fails a
saving throw vs. spell.\n\n

Notes: Common spell (PHB)."""

In [ ]:
found = re.findall('[a-zA-Z,]\n[a-zA-Z]', test)
repl = [x.replace('\n', ' ') for x in found]
z = list(zip(found, repl))

In [ ]:
for x, y in z:
    test = re.sub(x, y, test)

In [ ]:
for x in range(2):
    test = re.sub('\\n\\n\\n', '\n\n', test, count=0)

In [ ]:
re.findall('\n\n\n', test)

In [ ]:
test

## Read File into Dict

In [ ]:
d = list()
spell = dict()
with open("mem_spell.txt", encoding='utf-8') as f:
    for line in f:
        if line != '\n':
            print(line)
            (key, val) = line.split('| ')
            val = val.strip()
            if key in ['Class', 'School', 'Components']:
                val = literal_eval(val)
            spell[key] = val
        else:
            spell['Description'] = spell['Description'].replace('\\n', '\n')
            spell['Level'] = int(spell['Level'])
            d.append(spell)
            with open('spells2.json', 'a', encoding='utf-8', newline='\n') as fp:
                json.dump(spell, fp)
                fp.write('\n')
            spell = dict()

In [ ]:
d